In [41]:
from split import stratify
import pandas as pd
import numpy as np

In [44]:
df = pd.read_csv("data/intersection_files.csv")
data_df = df.copy()
df.head()

,smiles,B_Arrestin,ERK,G_Protein,Selective
0,C(CCOC1=CC2=CC=NN2C=C1)CNC1CCC2=CC=CC=C2C1,1,0,1,0
1,C1=NN2C=CC=CC2=C1,1,1,1,1
2,C1CCC2=CC=CC=C2C1,1,0,1,0
3,C(CCOC1=CN2N=CC=C2C=C1)CNC1CCC2=CC=CC=C2C1,1,0,1,0
4,C(C1=CNC2=CC=CC=C12)C1=CNC2=CC=CC=C12,1,0,1,0


In [45]:
for i in range(1, len(df.columns)):
    df[df.columns[i]] = df[df.columns[i]].apply(lambda x: df.columns[i] if x==1 else None)

In [46]:
df.head()

,smiles,B_Arrestin,ERK,G_Protein,Selective
0,C(CCOC1=CC2=CC=NN2C=C1)CNC1CCC2=CC=CC=C2C1,B_Arrestin,None,G_Protein,None
1,C1=NN2C=CC=CC2=C1,B_Arrestin,ERK,G_Protein,Selective
2,C1CCC2=CC=CC=C2C1,B_Arrestin,None,G_Protein,None
3,C(CCOC1=CN2N=CC=C2C=C1)CNC1CCC2=CC=CC=C2C1,B_Arrestin,None,G_Protein,None
4,C(C1=CNC2=CC=CC=C12)C1=CNC2=CC=CC=C12,B_Arrestin,None,G_Protein,None


In [47]:
data = [[i for i in d.tolist() if i is not None] for d in df.values]
classes = df.columns[1:].tolist()
split = stratify([i[1:] for i in data], classes, [0.8, 0.2])
train_indices = split[0][0]
test_indices = split[0][1]

In [48]:
data_df.iloc[train_indices].to_csv("train.csv", index=None)
data_df.iloc[test_indices].to_csv("test.csv", index=None)

In [80]:
actual = data_df.iloc[test_indices]
preds = pd.read_csv("preds.csv")
preds.head()

,smiles,B_Arrestin,ERK,G_Protein,Selective
0,O=C(CCCCCNC(=O)C1CC2=CC=CC=C2CN1)NCCC1=CC=CC=C1,0.589688,0.054574,0.524391,0.054794
1,C(CC1=CC=CC=C1)N1CCC(CC1)NC1=CC=CC=C1,0.491498,0.078000,0.610150,0.079413
2,C(CC1=CC=CC=C1)NCCC1=CC=CC=C1,0.416772,0.171605,0.756355,0.170457
3,C1C2=CC=CC=C2OC2=CC=CC=C12,0.242226,0.031142,0.889583,0.030767
4,C1CC[NH2+]CC1,0.910004,0.516363,0.904784,0.495300


In [82]:
convert = lambda x: 1 if float(x)>0.5 else 0

In [83]:
preds.iloc[:, 1:] = preds.iloc[:, 1:].applymap(convert)
preds.head()

,smiles,B_Arrestin,ERK,G_Protein,Selective
0,O=C(CCCCCNC(=O)C1CC2=CC=CC=C2CN1)NCCC1=CC=CC=C1,1,0,1,0
1,C(CC1=CC=CC=C1)N1CCC(CC1)NC1=CC=CC=C1,0,0,1,0
2,C(CC1=CC=CC=C1)NCCC1=CC=CC=C1,0,0,1,0
3,C1C2=CC=CC=C2OC2=CC=CC=C12,0,0,1,0
4,C1CC[NH2+]CC1,1,1,1,0


In [88]:
from sklearn.metrics import classification_report, multilabel_confusion_matrix

In [93]:
print(multilabel_confusion_matrix(actual.iloc[:, 1:].values, preds.iloc[:, 1:].values))

[[[ 94  14]
  [ 41  14]]

 [[140   2]
  [ 17   4]]

 [[ 30  29]
  [  7  97]]

 [[141   1]
  [ 16   5]]]
